In [ ]:
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import mediapipe as mp
import argparse
import imutils
import time
import dlib
import cv2

In [ ]:
def rect_to_bb(rect):
	# take a bounding predicted by dlib and convert it
	# to the format (x, y, w, h) as we would normally do
	# with OpenCV
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
	# return a tuple of (x, y, w, h)
	return (x, y, w, h)

In [ ]:
def eye_aspect_ratio(eye):
    # Compute the Euclidean distances between the two sets of vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # Compute the Euclidean distance between the horizontal eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
    # Compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    return ear

In [ ]:
def mouth_aspect_ratio(mouth):
    # Compute the Euclidean distances between the vertical mouth landmarks (x, y)-coordinates
    A = dist.euclidean(mouth[1], mouth[7])
    B = dist.euclidean(mouth[2], mouth[6])
    C = dist.euclidean(mouth[3], mouth[5])
    # Compute the Euclidean distance between the horizontal mouth landmarks (x, y)-coordinates
    D = dist.euclidean(mouth[0], mouth[4])
    # Compute the mouth aspect ratio
    mar = (A + B + C) / (2.0 * D)
    return mar

In [ ]:
# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 7

MOUTH_AR_THRESH = 0.3
MOUTH_AR_CONSEC_FRAMES = 30
# initialize the frame counters and the total number of blinks
ECOUNTER = 0
ETOTAL = 0
MCOUNTER = 0
MTOTAL = 0

In [ ]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

In [ ]:
# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["inner_mouth"]

In [ ]:
# # start the video stream thread
print("[INFO] starting video stream thread...")
# # vs = FileVideoStream(args["video"]).start()
# fileStream = True
# vs = VideoStream(src=0).start()
# # vs = VideoStream(usePiCamera=True).start()
# fileStream = False
# time.sleep(1.0)
# # loop over frames from the video stream

cap = cv2.VideoCapture(0)

while True:
    # if this is a file video stream, then we need to check if
    # there any more frames left in the buffer to process
#     if fileStream and not vs.more():
#         break
    success, image = cap.read()
    # grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale
    # channels)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # detect faces in the grayscale frame
    rects = detector(gray, 0)
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
    	# convert dlib's rectangle to a OpenCV-style bounding box
    	# [i.e., (x, y, w, h)], then draw the face bounding box
        (a, b, c, d) = face_utils.rect_to_bb(rect)
        cv2.rectangle(image, (a, b), (a + c, b + d), (0, 255, 0), 2)
        # extract the left and right eye coordinates, then use the
        # coordinates to compute the eye aspect ratio for both eyes
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        innerMouth = shape[mStart:mEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0
        mar = mouth_aspect_ratio(innerMouth)
        # compute the convex hull for the left and right eye, then
        # visualize each of the eyes
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        innerMouthHull = cv2.convexHull(innerMouth)
        cv2.drawContours(image, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(image, [rightEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(image, [innerMouthHull], -1, (0, 255, 0), 1)
        # check to see if the eye aspect ratio is below the blink
        # threshold, and if so, increment the blink frame counter
        if ear < EYE_AR_THRESH:
            ECOUNTER += 1
        # otherwise, the eye aspect ratio is not below the blink
        # threshold
        else:
            # if the eyes were closed for a sufficient number of
            # then increment the total number of blinks
            if ECOUNTER >= EYE_AR_CONSEC_FRAMES:
                ETOTAL += 1
            # reset the eye frame counter
            ECOUNTER = 0
        if mar > MOUTH_AR_THRESH:
            MCOUNTER += 1
        else:
            if MCOUNTER >= MOUTH_AR_CONSEC_FRAMES:
                MTOTAL += 1
            MCOUNTER = 0
        # draw the total number of blinks on the frame along with
        # the computed eye aspect ratio for the frame
        cv2.putText(image, "Blinks: {}".format(ETOTAL), (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(image, "Yawns: {}".format(MTOTAL), (10, 60),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(image, "EAR: {:.2f}".format(ear), (500, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(image, "MAR: {:.2f}".format(mar), (500, 60),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        # show the face number
        cv2.putText(image, "Face #{}".format(i + 1), (a - 10, b - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
    # show the frame
    cv2.imshow("Output", image)
    key = cv2.waitKey(1) & 0xFF
 
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
        
# do a bit of cleanup
cv2.destroyAllWindows()
cap.release()